In [1]:
### To use GPU
#from numba import jit, cuda

#to remove garbge
import gc

import numpy as np
import pandas as pd

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
from matplotlib.dates import drange
import datetime
import pyhht
#import emd
#from pyemd import emd

from scipy.signal import hilbert
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from scipy.io import loadmat

import plotly.express as px

# to filter(remove) the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_sbi = pd.read_excel("final_HDFCBANK_18-7_to_16-9-2022.xlsx") 
data_sbi['Date'] = pd.to_datetime(data_sbi['Date']).dt.date
c1 = data_sbi[['Date','Time','Close']]
del data_sbi
gc.collect()

109

In [3]:
#@jit(target_backend = 'cuda')

# def fl():
sum_acc_svr = 0
sum_acc_dtr = 0
sum_acc_rfr = 0

ss = 7000
svmd_ss = ss
n = 10000 - ss + 1

for i in range(n):
#     print('**********  For window - {} *********'.format(i+1))


#         d1 = c1[i: (ss)]
#         d2 = pd.read_excel('sbi__{}.xlsx'.format(i+1), header=None, names=['svmd1','svmd2','svmd3'])
    df = pd.concat([ (c1[i: (ss)]).reset_index(drop=True) , 
                    (pd.read_excel('hdfc__{}.xlsx'.format(i+1), header=None, names=['svmd1','svmd2','svmd3'])) ], 
                   axis=1)
    df = df[['Date','Time','svmd1','svmd2','svmd3','Close']]
    df.set_index(['Date','Time'], inplace=True)
#    print(df)

#         ## plotting decomposed modes
#         print('Plot of decomposed modes of the stock - SBIN - for each window-sized (7000) data ------->>>>')

#         df['svmd1'].plot(figsize = (55,13))
#         plt.ylabel('Mode 1 achieved by SVMD', fontsize=45)
#         plt.xlabel('Date, Time', fontsize=45)
#         plt.xticks(rotation='vertical',fontsize=40 )
#         plt.yticks(fontsize=40)
#         plt.margins(0.05)
#         plt.legend(fontsize=42)
#         plt.show()
#         df['svmd2'].plot(figsize = (55,13))
#         plt.ylabel('Mode 2 achieved by SVMD', fontsize=45)
#         plt.xlabel('Date, Time', fontsize=45)
#         plt.xticks(rotation='vertical',fontsize=40 )
#         plt.yticks(fontsize=40)
#         plt.margins(0.05)
#         plt.legend(fontsize=42)
#         plt.show()
#         df['svmd3'].plot(figsize = (55,13))
#         plt.ylabel('Mode 3 achieved by SVMD', fontsize=45)
#         plt.xlabel('Date, Time', fontsize=45)
#         plt.xticks(rotation='vertical',fontsize=40 )
#         plt.yticks(fontsize=40)
#         plt.margins(0.05)
#         plt.legend(fontsize=42)
#         plt.show()

#         ## 
#         s=0
#         res=[]
#         for j in range(svmd_ss):
#             s = df['svmd1'][j] + df['svmd2'][j] + df['svmd3'][j] 
#             res.append(s)
#         res_arr = np.array(res)
#         ###
#     #     print("&&&&&& length -> ",len(res_arr.transpose()), " and ", len(df.index))

#         svmd_close_recons = pd.DataFrame ( data=res_arr.transpose() , index = df.index , columns = ['Closing price (reconstructed by SVMD-modes)'])
#         svmd_recons_vs_close_df = svmd_close_recons.join(df.Close)
#         svmd_recons_vs_close_df.rename(columns={'Close':'Original Closing price'}, inplace=True)
    ### 
#         #print('Plot of Original vs Reconstructed signal for each window-sized data ***')
#         svmd_recons_vs_close_df.plot(figsize = (55,13))
#         plt.ylabel('Closing price', fontsize=45)
#         plt.xlabel('Date, Time', fontsize=45)
#         plt.xticks(rotation='vertical',fontsize=40 )
#         plt.yticks(fontsize=40)
#         plt.margins(0.05)
#         plt.legend(fontsize=42)
#         plt.show()

    ##############################################################
#     print("df----------->\n", df)
    X_train = df[['svmd1','svmd2','svmd3']]
    y_train = df[['Close']]

    ##$$$$
#     print(X_train)
#     print(y_train)


    # creating the SVR, DTR, RFR Models
    # SVR
    svr_rbf = SVR( kernel='rbf' )
    svr_rbf.fit ( X_train , y_train )
    # DTR
    DTR = DecisionTreeRegressor (random_state = 0)
    DTR.fit( X_train , y_train )
    # RFR
    N1 = int(np.sqrt(len(X_train)))
    RFR = RandomForestRegressor(n_estimators = N1 , random_state=0)
    RFR.fit(X_train , y_train )

    last = X_train.iloc[-1] ## the vey last row
    ## Predictions
    y_just_next_val_svr = svr_rbf.predict([last])[0]
    y_just_next_val_dtr = DTR.predict([last])[0]
    y_just_next_val_rfr = RFR.predict([last])[0]

    if i != (n-1):
        ## original given -->
        ori_giv = c1.iloc[ss]['Close']

        acc_svr = (1 - (abs(ori_giv - y_just_next_val_svr)/ori_giv))*100
        acc_dtr = (1 - (abs(ori_giv - y_just_next_val_dtr)/ori_giv))*100
        acc_rfr = (1 - (abs(ori_giv - y_just_next_val_rfr)/ori_giv))*100

#         print("By SVR,\nThe original given price is => ", ori_giv, "& predicted price is => ", y_just_next_val_svr,
#               "\n The accuracy is => ", acc_svr)
#         print("By DTR,\nThe original given price is => ", ori_giv, "& predicted price is => ", y_just_next_val_dtr,
#               "\n The accuracy is => ", acc_dtr)
#         print("By RFR,\nThe original given price is => ", ori_giv, "& predicted price is => ", y_just_next_val_rfr,
#               "\n The accuracy is => ", acc_rfr)
    elif i == (n-1):
        ori = c1.iloc[ss]['Close'] #original price of next day
        acc_svr = (1 - (abs(ori - y_just_next_val_svr)/ori))*100
        acc_dtr = (1 - (abs(ori - y_just_next_val_dtr)/ori))*100
        acc_rfr = (1 - (abs(ori - y_just_next_val_rfr)/ori))*100

        print("Total => n = ", n, "\n")
        print("By SVR,\nThe original given price is => ", ori, "& forecasted price is => ", y_just_next_val_svr,
              "\n The accuracy is => ", acc_svr)
        print("By DTR,\nThe original given price is => ", ori, "& forecasted price is => ", y_just_next_val_dtr,
              "\n The accuracy is => ", acc_dtr)
        print("By RFR,\nThe original given price is => ", ori, "& forecasted price is => ", y_just_next_val_rfr,
              "\n The accuracy is => ", acc_rfr)



    ### Total accuracy by 3 models separately----------------->
    sum_acc_svr += acc_svr
    sum_acc_dtr += acc_dtr
    sum_acc_rfr += acc_rfr


    ## inc
    ss = ss+1

    #### deleting for memory
    del svr_rbf, DTR, RFR, N1
    gc.collect()
    del df#, svmd_close_recons, svmd_recons_vs_close_df
    gc.collect()
    del X_train, y_train, acc_svr, acc_dtr, acc_rfr
    gc.collect()
    del y_just_next_val_svr, y_just_next_val_dtr, y_just_next_val_rfr
    gc.collect()

## Average Accuracy ---------------------------------------->>>>>>>>>>>>>>
avg_acc_svr = sum_acc_svr/n
avg_acc_dtr = sum_acc_dtr/n
avg_acc_rfr = sum_acc_rfr/n

print("The average accuracy by SVR => ", avg_acc_svr, "%")
print("The average accuracy by DTR => ", avg_acc_dtr, "%")
print("The average accuracy by RFR => ", avg_acc_rfr, "%")




Total => n =  3001 

By SVR,
The original given price is =>  1482.5 & forecasted price is =>  1470.0022406683693 
 The accuracy is =>  99.15698082080063
By DTR,
The original given price is =>  1482.5 & forecasted price is =>  1482.5 
 The accuracy is =>  100.0
By RFR,
The original given price is =>  1482.5 & forecasted price is =>  1482.5331325301202 
 The accuracy is =>  99.99776509071701
The average accuracy by SVR =>  97.84963211041038 %
The average accuracy by DTR =>  99.95972120146583 %
The average accuracy by RFR =>  99.95763306382031 %
